# Predict Emotion

The main objective of this notebook is to predict emotions from tweets

In [1]:
# Add project path to the PYTHONPATH

import os
import sys
from pathlib import Path

sys.path.append(Path(os.path.join(os.path.abspath(''), '../')).resolve().as_posix())

In [2]:
import pickle

## Load Tokenizer

Load `.pickle` file with the tokenizer

In [3]:
tokenizer_path = Path('../datasets/sentiment_analysis/tokenizer.pickle').resolve()
with tokenizer_path.open('rb') as file:
    tokenizer = pickle.load(file)

## Load Model

Load the trained emotion recognition model

In [4]:
from emotion_recognition.models.lstm_conv_model import lstm_conv_model

In [5]:
model_weights_path = Path('../models/emotion_recognition/model_weights.h5').resolve()
input_dim = min(tokenizer.num_words, len(tokenizer.word_index) + 1)
model = lstm_conv_model(100, input_dim, 4, embedding_dim=500)
model.load_weights(model_weights_path.as_posix())

W0718 13:36:48.834245 4427236800 deprecation.py:506] From /Users/rmohashi/miniconda3/envs/emodata/lib/python3.6/site-packages/tensorflow/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0718 13:36:48.923536 4427236800 deprecation.py:506] From /Users/rmohashi/miniconda3/envs/emodata/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0718 13:36:48.933838 4427236800 deprecation.py:506] From /Users/rmohashi/miniconda3/envs/emodata/lib/python3.6/site-packages/tensorflow/python/ops/init_op

## Load data

Load the data that will have the labels predicted by the model

**data_path**: Path to the `.csv` file that will be used

In [6]:
import pandas as pd

In [7]:
data_path = Path('../datasets/predict/1151893341782585349-1151863653320159233_kyoto_animation.csv').resolve()
data = pd.read_csv(data_path)
data.head()

,id,date,user,text
0,1151893341782585349,2019-07-18 16:35:48,Ozzzylot,⚡️ Fans share what Kyoto Animation studio mean...
1,1151893322908102657,2019-07-18 16:35:43,rosyutori,Deep condolences to all who are passed away at...
2,1151893318101377024,2019-07-18 16:35:42,met_bit,Striking news... How on earth can someone be s...
3,1151893304117813248,2019-07-18 16:35:39,Destructo_Dan,I don’t know if I had any favorite anime from ...
4,1151893302863650816,2019-07-18 16:35:39,KDiscavage,The news about Kyoto Animation Studios hit me ...


## Load Encoder

Load `.pickle` file with the encoder

In [8]:
encoder_path = Path('../models/emotion_recognition/encoder.pickle').resolve()
with encoder_path.open('rb') as file:
    encoder = pickle.load(file)

## Preprocess data

Preprocess the data that will be used

In [9]:
from nlp.utils import preprocess
from tensorflow.keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rmohashi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
cleaned_data = preprocess(data.text)
sequences = [text.split() for text in cleaned_data]
list_tokenized = tokenizer.texts_to_sequences(sequences)
x_data = pad_sequences(list_tokenized, maxlen=100)

Time to clean up: 1.15 sec


## Results

Predict the labels and generate a confusion matrix

In [11]:
import numpy as np

In [12]:
y_pred = model.predict(x_data)

In [13]:
for index, value in enumerate(np.sum(y_pred, axis=0) / len(y_pred)):
    print(encoder.classes_[index] + ": " + str(value))

angry: 0.0977998
fear: 0.3991122
joy: 0.03104621
sadness: 0.4720413


In [14]:
y_pred_argmax = y_pred.argmax(axis=1)
data_len = len(y_pred_argmax)
for index, value in enumerate(np.unique(y_pred_argmax)):
    print(encoder.classes_[index] + ": " + str(len(y_pred_argmax[y_pred_argmax == value]) / data_len))

angry: 0.09889558232931726
fear: 0.4011044176706827
joy: 0.030622489959839357
sadness: 0.46937751004016065


In [15]:
y_pred[5:10].argmax(axis=1)

array([3, 3, 3, 3, 3])

In [16]:
data.text.iloc[6]

'My heart goes out to the people who died in the fire at Kyoto Animation Studio. \n\n#PrayForKyoani https://t.co/Jvg9R8f6Oc'